In [5]:
import pandas as pd
import numpy as np

def load_data(sheet_name):
    data_file = "malformation_data.ods"
    df = pd.read_excel(data_file, sheet_name=sheet_name, index_col=0)
    return df

def statistic(N1, N2):
    """
    given two arrays N1 = (a1, b1, c1), N2 = (a2, b2, c2)
    return the sum of absolute differences of the proportions:
    |a1/(a1+b1+c1) - a2/(a2+b2+c2)| + |b1/(a1+b1+c1) - b2/(a2+b2+c2)| + |c1/(a1+b1+c1) - c2/(a2+b2+c2)|
    """
    P1 = N1 / N1.sum()
    P2 = N2 / N2.sum()
    return np.abs(P1 - P2).sum()

def resample(Ntot, n1):
    """
    given an array Ntot = (a, b, c) and an integer n1
    return two arrays N1 and N2 such that N1.sum() == n1 and N2.sum() == Ntot.sum() - n1,
    and N1 + N2 == Ntot
    """
    x = np.random.choice(range(sum(Ntot)), n1, replace=False)
    n_thr = 0
    N1 = np.zeros_like(Ntot)
    for i, n in enumerate(Ntot):
        N1[i] = ((n_thr <= x) & (x < n_thr + n)).sum()
        n_thr += n
    return N1, Ntot - N1


def calculate_pvalue(df, cond_1, cond_2, n_resamples):
    """
    given a dataframe df, two conditions cond_1 and cond_2, and an integer n_resamples
    return the p-value of the hypothesis that the two conditions have the same distribution.

    The p-value is calculated by pooling the two conditions together,
    resampling the pooled data n_resamples times keeping the relative number of observations constant,
    and estimating the empirical probability of observing a statistic at least as extreme as the one observed.
    """
    
    N1 = df.loc[cond_1].values
    N2 = df.loc[cond_2].values
    Ntot = N1 + N2

    s0 = statistic(N1, N2)

    n_fail = 1 # set to one to avoid zero
    for i in range(n_resamples):
        N1_res, N2_res = resample(Ntot, N1.sum())
        if statistic(N1_res, N2_res) >= s0:
            n_fail += 1
    return n_fail / (n_resamples + 1)

## calculate p-values for heart looping

In [6]:
sheet_name, wt_exp = "hearts", "2dpf_wt"
df = load_data(sheet_name)
df

,D-loop,mild/no loop,S-loop
experiment,,,
2dpf_wt,107,14,0
2dpf_triplMut,50,34,47
2dpf_metrnl1-/-,30,54,20
2dpf_metrn-/-,82,39,11
2dpf_triplMut_metrnl1 mRNA,60,85,15
2dpf_ triplMut_metrn mRNA,71,66,19
2dpf_ triplMut _metrnl1+metrn mRNA,85,44,33
2dpf_triplMut+/-,155,26,1
metrnlb-/-,77,22,5


In [7]:
np.random.seed(0)
n_resamples = int(1e5)

print(f"sheet name: {sheet_name}")
print(f"wild-type experiment: {wt_exp}")
print(f"number of resamples: {n_resamples}")
print("------")

pval_dict = {}
for exp in df.index.to_list():
    print(f"processing {wt_exp} vs {exp}")

    pval_dict[exp] = calculate_pvalue(df, wt_exp, exp, n_resamples)
    print(f"pval <= {pval_dict[exp]:.2}")

df[f"pval_{wt_exp}"] = df.index.map(pval_dict)
# save results
wt_lab = wt_exp.replace("/", "")
df.to_csv(f"results/pval_{sheet_name}_{wt_lab}.csv")

df


sheet name: hearts
wild-type experiment: 2dpf_wt
number of resamples: 100000
------
processing 2dpf_wt vs 2dpf_wt
pval <= 1.0
processing 2dpf_wt vs 2dpf_triplMut
pval <= 1e-05
processing 2dpf_wt vs  2dpf_metrnl1-/-
pval <= 1e-05
processing 2dpf_wt vs 2dpf_metrn-/-
pval <= 1e-05
processing 2dpf_wt vs 2dpf_triplMut_metrnl1 mRNA
pval <= 1e-05
processing 2dpf_wt vs 2dpf_ triplMut_metrn mRNA
pval <= 1e-05
processing 2dpf_wt vs 2dpf_ triplMut _metrnl1+metrn mRNA
pval <= 1e-05
processing 2dpf_wt vs 2dpf_triplMut+/-
pval <= 0.49
processing 2dpf_wt vs metrnlb-/-
pval <= 0.007


,D-loop,mild/no loop,S-loop,pval_2dpf_wt
experiment,,,,
2dpf_wt,107,14,0,1.000000
2dpf_triplMut,50,34,47,0.000010
2dpf_metrnl1-/-,30,54,20,0.000010
2dpf_metrn-/-,82,39,11,0.000010
2dpf_triplMut_metrnl1 mRNA,60,85,15,0.000010
2dpf_ triplMut_metrn mRNA,71,66,19,0.000010
2dpf_ triplMut _metrnl1+metrn mRNA,85,44,33,0.000010
2dpf_triplMut+/-,155,26,1,0.491505
metrnlb-/-,77,22,5,0.007030


## calculate p-values for dand5

In [8]:
sheet_name, wt_exp = "dand5", "14hpf_wt"
df = load_data(sheet_name)
df

,normal,disturbed,reduced
experiment,,,
14hpf_wt,97,18,3
14hpf_triplMut,20,31,83
tripleMut+/-,77,10,40
metrn-/-,47,48,34
metrnl1-/-,84,33,41


In [9]:
np.random.seed(0)
n_resamples = int(1e5)

print(f"sheet name: {sheet_name}")
print(f"wild-type experiment: {wt_exp}")
print(f"number of resamples: {n_resamples}")
print("------")

pval_dict = {}
for exp in df.index.to_list():
    print(f"processing {wt_exp} vs {exp}")

    pval_dict[exp] = calculate_pvalue(df, wt_exp, exp, n_resamples)
    print(f"pval <= {pval_dict[exp]:.2}")

df[f"pval_{wt_exp}"] = df.index.map(pval_dict)
# save results
wt_lab = wt_exp.replace("/", "")
df.to_csv(f"results/pval_{sheet_name}_{wt_lab}.csv")

df

sheet name: dand5
wild-type experiment: 14hpf_wt
number of resamples: 100000
------
processing 14hpf_wt vs 14hpf_wt
pval <= 1.0
processing 14hpf_wt vs 14hpf_triplMut
pval <= 1e-05
processing 14hpf_wt vs tripleMut+/-
pval <= 1e-05
processing 14hpf_wt vs metrn-/-
pval <= 1e-05
processing 14hpf_wt vs metrnl1-/-
pval <= 1e-05


,normal,disturbed,reduced,pval_14hpf_wt
experiment,,,,
14hpf_wt,97,18,3,1.00000
14hpf_triplMut,20,31,83,0.00001
tripleMut+/-,77,10,40,0.00001
metrn-/-,47,48,34,0.00001
metrnl1-/-,84,33,41,0.00001


## calculate p-values for organ positioning

In [10]:
sheet_name, wt_exp = "gata6", "54hpf_wt"
df = load_data(sheet_name)
df

,normal,strait/bilateral,reverse
experiment,,,
54hpf_wt,137,7,2
54hpf_triplMut,129,17,16


In [11]:
np.random.seed(0)
n_resamples = int(1e5)

print(f"sheet name: {sheet_name}")
print(f"wild-type experiment: {wt_exp}")
print(f"number of resamples: {n_resamples}")
print("------")

pval_dict = {}
for exp in df.index.to_list():
    print(f"processing {wt_exp} vs {exp}")

    pval_dict[exp] = calculate_pvalue(df, wt_exp, exp, n_resamples)
    print(f"pval <= {pval_dict[exp]:.2}")

df[f"pval_{wt_exp}"] = df.index.map(pval_dict)
# save results
wt_lab = wt_exp.replace("/", "")
df.to_csv(f"results/pval_{sheet_name}_{wt_lab}.csv")

df

sheet name: gata6
wild-type experiment: 54hpf_wt
number of resamples: 100000
------
processing 54hpf_wt vs 54hpf_wt
pval <= 1.0
processing 54hpf_wt vs 54hpf_triplMut
pval <= 0.00043


,normal,strait/bilateral,reverse,pval_54hpf_wt
experiment,,,,
54hpf_wt,137,7,2,1.00000
54hpf_triplMut,129,17,16,0.00043
